## Winters, Alexander (V00970263)

# Problem 2. Decisions

### Sources:

https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4

https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f

https://en.wikipedia.org/wiki/Decision_tree_learning

https://en.wikipedia.org/wiki/ID3_algorithm

https://www.youtube.com/watch?v=UdTKxGQvYdc

In [1]:
import numpy as np
np.random.seed(1337)

In [2]:
import pandas as pd
# Plotting support
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
elections_df = pd.read_csv('elections_clean.csv', index_col=False)
elections_df.columns

Index(['votes', 'unemployment', 'med_hh_inc', 'per_capita_inc',
       'poverty_all_ages', 'deep_pov_all', 'deep_pov_children', 'population',
       'total_area', 'pop_density', 'total_male', 'total_female',
       'voter_turnout', 'democrat', 'county', 'state', 'education', 'religion',
       'age_young', 'age_adult', 'age_old', 'ethnic_male', 'ethnic_female'],
      dtype='object')

In [4]:
# Place the label vector at the end of the df
label_vector = elections_df.pop('democrat')
elections_df['democrat'] = label_vector

# Take only the categorial features
categorial_features = ['education', 'religion', 'ethnic_male', 'ethnic_female']

In [5]:
# Calculate the entropy of the whole dataset
def dataset_entropy(elections_data, attribute, attribute_list):
    entropy_total = 0
    
    for value in attribute_list:
        prob = elections_data[attribute].value_counts()[value] / elections_data[attribute].shape[0]
        entropy_total += - prob * np.log2(prob)
 
    return entropy_total

In [6]:
# Function to calculate entropy of each feature

def entropy(elections_data, feature_name, attribute, attribute_list):
    attribute_entropy = 0
    variables = elections_data[feature_name].unique()
    
    for variable in variables:
        feature_entropy = 0
        
        for target_value in attribute_list:
            numerator = len(elections_data[feature_name][elections_data[feature_name] == variable][elections_data[attribute] == target_value])
            denominator = len(elections_data[feature_name][elections_data[feature_name] == variable])
            frac = numerator / (denominator + np.finfo(float).eps)
            feature_entropy += - frac * np.log2(frac + np.finfo(float).eps)
        
        frac2 = denominator / elections_data.shape[0]
        attribute_entropy += -frac2 * feature_entropy
    
    return abs(attribute_entropy)

In [7]:
# Determines information gain of each feature

def information_gain(elections_data, feature_name, attribute, attribute_list):
    return (dataset_entropy(elections_data, attribute, attribute_list) - entropy(elections_data, feature_name, attribute, attribute_list))

In [8]:
# Returns feature with max information gain
def max_info_gain(elections_data, attribute, attribute_list, categorial_features):
    max_feature_gains = {}
    
    for feature in categorial_features:
        feature_ig = information_gain(elections_data, feature, attribute, attribute_list)
        max_feature_gains[feature] = feature_ig
        
    return max(max_feature_gains, key=max_feature_gains.get)

In [9]:
def ID3(elections_data, attribute, categorial_features, parent=None):
    # Check if label vector is pure and return it to go back up the tree
    if len(np.unique(elections_data[attribute])) <= 1:
        return np.unique(elections_data[attribute])[0]
    
    # Check if we reached the end of the branch which returns the parent and goes up the tree
    if len(categorial_features) == 0:
        return parent
    
    best_feature = max_info_gain(elections_data, attribute, attribute_list, categorial_features)
    best_feature_values, value_counts = np.unique(elections_data[best_feature], return_counts=True)
    # Make the parent node the most common element
    # Gets rid of values equal to 'None'
    parent = elections_data[attribute].mode()[0]
    
    tree = {best_feature:{}}
    # Remove the best feature from our feature list
    feats = [i for i in categorial_features if i != best_feature]

    for value in best_feature_values:
        subset = elections_data[elections_data[best_feature] == value].reset_index(drop=True)
        tree[best_feature][value] = ID3(subset, attribute, feats, parent)

    return tree
            

In [10]:
attribute = 'democrat'
elections_data = elections_df.copy()
attribute_list = elections_data[attribute].unique()

In [11]:
# Shuffle data for 70/30 split into training and validation sets
shuffled_dataset = elections_df.sample(frac=1).reset_index(drop=True)
split = int(elections_df.shape[0] * 0.7)

X_train = shuffled_dataset.iloc[:split].reset_index(drop=True)
X_test = shuffled_dataset.iloc[split:].reset_index(drop=True)

In [12]:
def predict(tree, instance):
    # Check if tree is dictionary
    if not isinstance(tree, dict):
        return tree
    else:
        # Make the 'root' the next element of the tree
        root = next(iter(tree))
        feature = instance[root]
        
        if feature in tree[root]:
            return predict(tree[root][feature], instance)
        else:
            return None

In [13]:
def validate(tree, validation_set):
    correct = 0
    wrong = 0
    
    for index, row in validation_set.iterrows():
        result = predict(tree, validation_set.iloc[index])
        if result == validation_set['democrat'].iloc[index]:
            correct += 1
        else:
            wrong += 1
            
    accuracy = correct / (correct + wrong) * 100
    return accuracy    

In [14]:
DTree = ID3(X_train, attribute, categorial_features)
train_acc = validate(DTree, X_train)
train_err = 100 - train_acc

print("The training accuracy of the Decision-Tree: " + str(train_acc) + " %")
print("The training error of the Decision-Tree: " + str(train_err) + " %\n")

test_acc = validate(DTree, X_test)
test_err = 100 - test_acc

print("The validation accuracy of the Decision-Tree: " + str(test_acc) + " %")
print("The validation error of the Decision-Tree: " + str(test_err) + " %\n")
print()

The training accuracy of the Decision-Tree: 89.91367560199909 %
The training error of the Decision-Tree: 10.086324398000912 %

The validation accuracy of the Decision-Tree: 87.07627118644068 %
The validation error of the Decision-Tree: 12.923728813559322 %




In [15]:
import pprint
pprint.pprint(DTree)

{'ethnic_male': {'Asian Male': {'religion': {'Catholic': {'education': {"Bachelor's Degree or Higher": 0,
                                                                        'Less than High School Diploma': 0,
                                                                        'Only High School Diploma': {'ethnic_female': {'Black Female': 0,
                                                                                                                       'Native American Female': 0}},
                                                                        "Some College or Associate's Degree": {'ethnic_female': {'Black Female': 0,
                                                                                                                                 'Native American Female': 0}}}},
                                             'Christian Generic': {'education': {"Bachelor's Degree or Higher": 0,
                                                                         

In [16]:
def num_stumps(DTree,  stumps = {}): 
    for feature in list(DTree):
        # Check if stump is a leaf node or sub tree
        stump = DTree[feature]
        
        # If not a leaf node, check if stump has been seen before and add to count
        if stump != 1 and stump != 0:
            if feature in stumps:
                stumps[feature] += 1
            else:
                stumps[feature] = 1
            
            num_stumps(stump, stumps)
        
    return stumps

In [17]:
stumps = num_stumps(DTree)
print("The features in the Decision-Tree and their repitition in decision stumps:\n")
for stump in stumps:
    print(stump + ": " + str(stumps[stump]))

The features in the Decision-Tree and their repitition in decision stumps:

ethnic_male: 1
Asian Male: 1
religion: 12
Catholic: 5
education: 20
Only High School Diploma: 9
ethnic_female: 44
Some College or Associate's Degree: 6
Christian Generic: 3
Mainline Christian: 8
Bachelor's Degree or Higher: 6
Less than High School Diploma: 7
NON-Catholic CHRISTIAN: 4
Other: 7
Other MISC: 3
Pentecostal / Charismatic: 3
Black Male: 1
Black Female: 2
White Female: 3
Mormon: 2
Other CHRISTIAN: 1
Multi Male: 1
Native American Male: 1
Amish: 1
Native American Female: 1
White Male: 1


In [18]:
def tree_depth(DTree, prev_depth, stumps = {}):
    curr_depth = prev_depth
    
    for feature in list(DTree):
        # Check if stump is a leaf node or sub tree
        stump = DTree[feature]
        # If not a leaf node, add to depth count
        if stump != 1 and stump != 0:
            depth = tree_depth(stump, prev_depth + 1, stumps)
            # Take the larger depth of our stumps
            curr_depth = max(depth, curr_depth)

    return curr_depth

In [19]:
max_depth = tree_depth(DTree, 1)
print("Maximum Depth of Decision-Tree is: " + str(max_depth))

Maximum Depth of Decision-Tree is: 8
